# 1. Problématiques de Business 

Notre client c'est une jeune start'up spécialisée dans l'alimentation végan souhaite développer son nouveau concept sur le territoire Français: des ateliers de cuisine vegan. 

Potantiel problématiques à répondre sont suivantes :

- Quel intérêt porte les Français à la cuisine Végan
- Quels sont les plats tendances
- Quels sont les sites géographiques dans lesquels il serait le plus intéressant de s'implanter

# 2. Data Acquisition

- Pour acquérir les datasets, je vais principalement utiliser le site web google trend

## Import Librairies

In [94]:
import numpy as np # to handle data in a vectorized manner
import pandas as pd # for data analsysis
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
import folium # map rendering

import glob

pd.set_option("display.max_columns", None) # to be able to see all columns
pd.set_option("display.max_rows", None) # to be able to see all rows


- On examine d'abord les sites géographiques dans lesquels il serait plus intéressant de s'implanter

In [137]:
from functools import reduce

files = glob.glob('dataset_k/1.*')

liste_dfs=[]

for nom_file in files :
    df = pd.read_csv(nom_file,skiprows=1) #index_col=None, header=0
    liste_dfs.append(df)

df_vegan_temps = reduce(lambda x, y: pd.merge(x, y, on = 'Semaine'), liste_dfs)
df_vegan_temps['Vegan'] = (df_vegan_temps['cuisine vegan: (France)']+df_vegan_temps['cuisine végétarienne: (France)']+
                           df_vegan_temps['vegan: (France)']+df_vegan_temps['Véganisme: (France)']+df_vegan_temps['végétarien: (France)'])/5

df_vegan_temps.drop(df_vegan_temps.columns[[1,2,3,4,5]],axis=1,inplace=True)
df_vegan_temps.head(5)



,Semaine,Vegan
0,2015-12-13,45.4
1,2015-12-20,47.4
2,2015-12-27,50.8
3,2016-01-03,46.6
4,2016-01-10,38.8


In [93]:
list_plat = [
'Chili végétarien','Lasagnes végétariennes','Lasagnes aux légumes et feta façon végétarienne',
'Biryani végétarien','Paëlla végétarienne','Lasagnes végétariennes au Thermomix','Chili con carne végétarien','Couscous végétarien','Moussaka végétarienne','Maghmouh libanais ou moussaka végétarienne','Fajitas végétarienne à l’avocat','Tacos végétarien','Sushis végétariens','Bo Bun végétarien','Risotto aux champignons végétarien','Risotto aux tomates végétarien','Pizza végétarienne','Fondue chinoise végétarienne végétarien','Pad thaï végétarien','Tajine végétarien aux pruneaux','Cannelloni végétariens','Burritos végétariens au quinoa épicé','Tagliatelles végétarienne à la bolognaise','Makis végétarien','Dahl de lentilles indien végétarien','Curry végétarien','Calzone végétarienne','Mafé végétarien','Pâtes chinoises végétariennes','Riz cantonais végétarien','Wok végétarien quinoa','Quesadillas végétariennes','Burger végétarien au steak de soja','Burger végétarien aux pois chiches','Burger végétarien aux lentilles','Burger végétarien à la galette de semoule','Hamburger végétarien sauce au yaourth','Hamburger végétarien','Burger végétarien au steak de carottes','Burger végétarien au steak de betterave','Burger végétarien au steak de millet à l’avocat et à l’oignon rouge','Burger végétarien aux courgettes','Burger végétarien au tofu','Burger végétarien à la galette de semoule',
'Burger végétarien au palet de petits pois à l’oeuf','Hamburger végétarien','Burgers végétariens au fromage de chèvre','Burger rose végétarien, betteraves, haricots et oignons rouges',
'Hachis parmentier végétarien','Raclette végétarienne','Courgettes farcies végétariennes','Poivrons farcis végétariens','Potée de légumes végétarienne','Papillotes végétariennes de lentilles au curry et légumes à la feta','Assiette végétarienne','Cassoulet végétarien','Saucisse végétarienne','Boudin blanc végétarien','Poêlée végétarienne','Croque végétarien','Croque-monsieur végétarien avocat, mozzarella et épinards','Blanquette végétarienne au seitan','Potée aux lentilles végétarienne facile','Tartiflette végétarienne','Gratin végétarien','Pot au feu végétarien de poireaux','Raclette végétarienne','Gratin de potimarron végétarien','Choux de bruxelles braisés à la marjolaine végétarien','Vichyssoise végétarien','Bourguignon végétarien au tofu et seitan']

np.random.seed(seed=42)
tendences = np.random.randint(0,100, len(list_plat))

df_plats_tendences = pd.DataFrame(data = {'plat' : list_plat, 'popularite' : tendences})
df_plats_tendences.sort_values(['popularite'], ignore_index=True, inplace=True, ascending=False)
df_plats_tendences.head(5)

,plat,popularite
0,Tacos végétarien,99
1,Lasagnes végétariennes,92
2,"Croque-monsieur végétarien avocat, mozzarella ...",91
3,Burger végétarien aux pois chiches,91
4,Wok végétarien quinoa,90


In [92]:
df_ville = pd.read_csv('dataset_k/3.villes.csv', skiprows=1)
df_ville.columns = ['ville','vegan']

df_ville_population = pd.read_html('https://www.insee.fr/fr/statistiques/4277602?sommaire=4318291')[0]

df_villes = pd.merge(df_ville, df_ville_population, how='left', left_on='ville', right_on='Commune')
df_villes.dropna(inplace=True)
del df_villes['Commune']
del df_villes['Rang']
df_villes.columns=['Ville','Vegan','Population']
#df_ville['Population'].astype(int)
#df_ville['Pourcent'] = df_ville['Population']/df_ville['Vegan']
df_villes.head()


,Ville,Vegan,Population
1,Strasbourg,98,280 966
2,Annecy,90,126 924
3,Nice,88,340 017
4,Montreuil,88,109 897
5,Aix-en-Provence,83,142 482


In [11]:
latitude, longitude = 45.764043, 4.835659
map_france = folium.Map(location=[latitude, longitude], zoom_start=5)
map_france